In [1]:
import os

os.chdir('../')
os.getcwd()

'/media/nazirnayal/DATA/projects/attention_long_tail/paying-attention-to-the-long-tail'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import wandb
import pytorch_lightning as pl
import yaml
from transformer_pl import Transformer
from pytorch_lightning.loggers import WandbLogger
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def read_config(path):
    with open(path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config


config = read_config('configs/classification/cifar10_config_local.yaml')



if isinstance(config['class_names'], str):
    class_names = read_config(config['class_names'])
    config['class_names'] = list(class_names.values())

In [7]:
# read checkpoints paths:
model_dirs = os.listdir('model_logs')

for model_dir in model_dirs:
    
    if 'cifar100' in model_dir:
        continue
    
    
    model_path = os.path.join('model_logs', model_dir)
    ckpt_path = os.listdir(model_path)[0]
    ckpt_path = os.path.join(model_path, ckpt_path)
    
    if 'CB' in model_dir:
        config['loss_function'] = 'class_balanced_loss'
        if 'CE' in model_dir:
            config['class_balanced_loss_type'] = 'cross_entropy'
        else:
            config['class_balanced_loss_type'] = 'focal_loss'
    elif 'WCE' in model_dir:
        config['loss_function'] = 'weighted_cross_entropy'
    elif 'CE' in model_dir:
        config['loss_function'] = 'cross_entropy'
    else:
        config['loss_function'] = 'focal_loss'
    
    if 'accval' in model_dir:
        run_name = model_dir[:-11]
    else:
        run_name = model_dir[:-4]
        
    model = Transformer(config)
    wandb_logger = WandbLogger(name=run_name, project='attention_LT_cifar10_test', job_type='train', log_model=True)
    outputs = pl.Trainer(
        gpus=1, 
        profiler='simple',
        logger=wandb_logger
    ).test(
        model, 
        ckpt_path=ckpt_path
    )            
    
    wandb.finish()
    
    


KeyboardInterrupt: 